# Music Discovery AI Agent

## Setup and installation

In [1]:
# CELL 1: MINIMAL SETUP
!pip install langgraph requests

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.8 MB/s eta 0:00:00
Mounted at /content/drive


## Configuration and imports

In [2]:
# CELL 2: CONFIGURATION AND IMPORTS
import os
import sqlite3
import requests
import json
import re
import sys
from typing import TypedDict, List, Dict, Any, Tuple
from langgraph.graph import StateGraph, END
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Set up paths
DRIVE_PATH = "/content/drive/MyDrive/Colab-Notebooks/last-fm-data"
DATA_PATH = os.path.join(DRIVE_PATH, "data")
DB_PATH = os.path.join(DATA_PATH, "lastfm_data.db")

# IMPORTANT: Add to sys.path BEFORE importing
# Python can't find the utils directory if the path isn't in sys.path
sys.path.append(DRIVE_PATH)

# Import config
try:
    from config.config import LASTFM_API_KEY, USERNAME
    print(f"Config loaded - User: {USERNAME}")
    lastfm_api_key = LASTFM_API_KEY
except ImportError:
    print("No config found")
    lastfm_api_key = None

# Create directories and test database
os.makedirs(DATA_PATH, exist_ok=True)
print(f"Database: {os.path.exists(DB_PATH)}")

Config loaded - User: emmenru
Database: True


## Init Helpers: ....

In [3]:
# CELL 3: INITIALIZE HELPERS
from utils.music_discovery_helper import (
    AgentState, DatabaseManager, HuggingFaceHelper,
    LastFMAPIHelper, extract_entities, classify_intent_node_hf,
    generate_artist_info_query, generate_recommendations_query,
    generate_genre_exploration_query, generate_listening_analysis_query,
    execute_database_query, call_lastfm_api, generate_response_hf
)

# Initialize database and load artist names
db_manager = DatabaseManager(DB_PATH)
db_manager.test_connection()
artist_names = db_manager.get_artist_names(200)

# Initialize HF helper
hf_helper = HuggingFaceHelper()
print("🤗 Helpers initialized")

✅ Database: 13 tables
📊 Loaded 200 artist names
🤗 Helpers initialized


## LangGraph Workflow Creation

In [4]:
# WORKFLOW CREATION CELL
from utils.music_discovery_helper import classify_intent_node_hf

def create_hf_music_agent():
    """Create the LangGraph workflow using Hugging Face"""

    # Create the state graph
    workflow = StateGraph(AgentState)

    # Add nodes with lambda functions to pass dependencies
    workflow.add_node("classify_intent", lambda state: classify_intent_node_hf(state, hf_helper, artist_names))
    workflow.add_node("artist_info_query", generate_artist_info_query)
    workflow.add_node("recommendations_query", generate_recommendations_query)
    workflow.add_node("genre_query", generate_genre_exploration_query)
    workflow.add_node("listening_query", generate_listening_analysis_query)
    workflow.add_node("execute_query", lambda state: execute_database_query(state, db_manager))
    workflow.add_node("api_call", lambda state: call_lastfm_api(state, lastfm_api_key))
    workflow.add_node("generate_response", generate_response_hf)

    # Define the workflow logic
    def route_after_intent(state: AgentState) -> str:
        """Route to appropriate query generation based on intent"""
        intent = state.get("intent", "")

        if intent == "ARTIST_INFO":
            return "artist_info_query"
        elif intent == "RECOMMENDATIONS":
            return "recommendations_query"
        elif intent == "GENRE_EXPLORATION":
            return "genre_query"
        elif intent == "LISTENING_ANALYSIS":
            return "listening_query"
        else:
            return "listening_query"  # Default fallback

    # Set up the workflow
    workflow.set_entry_point("classify_intent")

    workflow.add_conditional_edges(
        "classify_intent",
        route_after_intent,
        {
            "artist_info_query": "artist_info_query",
            "recommendations_query": "recommendations_query",
            "genre_query": "genre_query",
            "listening_query": "listening_query"
        }
    )

    # All query nodes lead to execution
    workflow.add_edge("artist_info_query", "execute_query")
    workflow.add_edge("recommendations_query", "execute_query")
    workflow.add_edge("genre_query", "execute_query")
    workflow.add_edge("listening_query", "execute_query")

    # After execution, potentially call API, then generate response
    workflow.add_edge("execute_query", "api_call")
    workflow.add_edge("api_call", "generate_response")
    workflow.add_edge("generate_response", END)

    return workflow.compile()

# Create the agent
print("🤗 Creating Hugging Face + LangGraph music discovery agent...")
agent = create_hf_music_agent()
print("✅ Agent created successfully!")

🤗 Creating Hugging Face + LangGraph music discovery agent...
✅ Agent created successfully!


## Interface

In [5]:
# CELL 5: SIMPLE CHAT
def ask_agent(query: str):
    """Simple function to ask the agent a question"""
    initial_state = {
        "user_query": query,
        "intent": "",
        "confidence": 0.0,
        "entities": [],
        "sql_query": "",
        "query_results": [],
        "api_results": [],
        "response": "",
        "error_message": "",
        "needs_api_call": False
    }

    try:
        result = agent.invoke(initial_state)

        if result.get("error_message"):
            print(f"❌ Error: {result['error_message']}")
        else:
            print("🎵 Response:")
            print(result["response"])

        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return {"error_message": str(e)}

# Usage:
ask_agent("What are my top 5 artists?")

HF API returned 401
🎯 Intent: LISTENING_ANALYSIS (confidence: 0.80)
🏷️ Entities: ['5']
📊 Analyzing your top 5 artists
📊 Query executed successfully. Found 5 results.
Sample results:
  1. ['artist_name', 'playcount', 'rank']...
  2. ['artist_name', 'playcount', 'rank']...
✅ Response generated successfully
🎵 Response:
🎵 **Your Top Artists:**

1. **Tomas Andersson Wij** - 1370 plays (seen live,indie,swedish,pop,scandinavian,Sweden,in...)
2. **The Radio Dept.** - 1145 plays (seen live,indie,swedish,Dreamy,pop,scandinavian,Sw...)
3. **M83** - 856 plays (seen live,indie,Dreamy,pop,chillout,electronic,ele...)
4. **Bob Dylan** - 811 plays (seen live,indie,pop,singer-songwriter,american,gen...)
5. **Håkan Hellström** - 755 plays (seen live,indie,swedish,Swedish Pop,pop,scandinavi...)



{'user_query': 'What are my top 5 artists?',
 'intent': 'LISTENING_ANALYSIS',
 'confidence': 0.8,
 'entities': ['5'],
 'sql_query': "\n    SELECT \n        a.name as artist_name,\n        uta.playcount,\n        uta.rank,\n        GROUP_CONCAT(DISTINCT t.name) as tags\n    FROM user_top_artists uta\n    JOIN artists a ON uta.artist_id = a.artist_id\n    LEFT JOIN artist_tags at ON a.artist_id = at.artist_id\n    LEFT JOIN tags t ON at.tag_id = t.tag_id\n    WHERE uta.time_period = 'overall'\n    GROUP BY a.artist_id, a.name, uta.playcount, uta.rank\n    ORDER BY uta.rank\n    LIMIT 5\n    ",
 'query_results': [{'artist_name': 'Tomas Andersson Wij',
   'playcount': 1370,
   'rank': 1,
   'tags': 'seen live,indie,swedish,pop,scandinavian,Sweden,indie pop,singer-songwriter,spotify,romantic,Awesome,00s,rock,alternative,male vocalists,svenskt,sett live,nordic,folk,favorites,emo,acoustic,stockholm,:To listen to again later:,christian,spiritual,guitar,folk-rock,singer,folk pop,singer songwrit